In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Read data
Finnis = pd.read_csv("../input/st-wave/Finnis Konect_FinnisSeaView_Wave.csv")
South_Hunter = pd.read_csv("../input/st-wave/South Hunter Konect_S_HuntSeaView_Wave.csv")
Splaugh = pd.read_csv("../input/st-wave/Splaugh Konect_SplaughSeaView_Wave.csv")
Finnis_test = pd.read_csv("../input/st-wave-test/Finnis Konect_FinnisSeaView_Wave_test.csv")
Finnis_test2 = pd.read_csv("../input/st-wave-test/Finnis Konect_FinnisSeaView_Wave_test2.csv")

# Test 9: Gap Test

In [3]:
def gap_test(data, N=4):
    #test 9
    # Calculate the number of cumulative missing values
    missing = data.isnull().astype(int).groupby(data.notnull().astype(int).cumsum()).cumsum()
    C2 = max(missing)
    print(C2)
    # Compare to limit N
    if C2>=N:
        flag = 4
    else:
        flag = 1
    return flag

In [4]:
print(gap_test(Finnis.SeaView_PSVSW_Hmax),gap_test(South_Hunter.SeaView_PSVSW_Hs),gap_test(Finnis_test.SeaView_PSVSW_Hs))

In [5]:
Finnis_test.SeaView_PSVSW_Hs

# Test 10: Spike Test

In [6]:
def spike_test(data, num_stdev=4):
    #test 10
    mean_value = data.mean()
    stdev_value = data.std()
    results = np.full(data.shape, 4)
    fail = abs(data - mean_value) > num_stdev * stdev_value
    results[~fail] = 1
    return results

In [7]:
spike_test(Finnis_test2.SeaView_PSVSW_DominantPeriod)

# Test 11: Range Test

In [8]:
def range_test(data, IMIN=-2000, IMAX=2000, LMIN=-1050, LMAX=1050):
    #test 11
    """
    NOTE: 
    * Default local limits are based on past 2 years of M6 data 
        from new buoy - maximum Hmax is ~21m.
    * Default instrument limits based on Datawell Heave range
        -20m to +20m
    """
    results = []
    for val in data:
        if (val>IMAX or val<IMIN):
            results.append(4)
        elif (val>LMAX or val<LMIN):
            results.append(3)
        else:
            results.append(1)
    return results

In [9]:
range_test(Finnis_test2.SeaView_PSVSW_DominantPeriod)

# Test 12: Segment Shift

In [10]:
def segment_shift(data,thresh = 0.2, n = 8):
    #test 12
    

    #first define the means
    length_of_segment = int(len(data)/n)
    for i in range(2,n+1):
        if abs(data[(i-2)*length_of_segment:(i-1)*length_of_segment].mean()-data[(i-1)*length_of_segment:i*length_of_segment].mean())>=thresh:
            print(4)
        else:
            print(1)
    
    #if abs(m[i]-m[i+1])<thresh:
    #    flag = 4
    #else:
    #    flag = 1
    #return flag

    
    

In [11]:
segment_shift(Finnis_test2.SeaView_PSVSW_DominantPeriod)

# Test 13: Acceleration Test

In [12]:
def acceleration_test(data, M = 0.5, G = 9.8, N = 10):
    counter = 0
    for i in range(len(data)):
        if data[i] > M*G:
            counter += 1
            flag = 3
            #interpolation part
            #WARNING!! this block overwrites data
            #try:
            #    data[i] = (data[i-1]+data[i+1])/2
            #except:
            #    pass
        else:
            flag = 1
            #print(counter, flag)
            #interpolate acc data
            
    if counter>N:
        flag = 4
    print('flag:',flag,'counter:',counter)
        

In [13]:
acceleration_test(Finnis_test2.SeaView_PSVSW_AccEarthFr_accN)

In [14]:
acceleration_test(Finnis_test2.SeaView_PSVSW_AVGPERIOD_TZ)

In [15]:
Finnis_test2.SeaView_PSVSW_AVGPERIOD_TZ